In [1]:
############### Import packages
import os, numpy as np, pandas as pd, time, glob, re, math, statsmodels.api as sm, patsy as ps
from tqdm import tqdm
from time import process_time
from datetime import datetime
from datetime import date
from openpyxl import load_workbook
from patsy import dmatrices

############### Set working directory to parent directory
if os.getcwd() != 'F:\\github\\narrative_conservatism\\code':
    os.chdir('F:\\github\\narrative_conservatism\\code')
    
############### Set pandas column printing constraint
pd.set_option('display.max_columns', None)

In [2]:
########################################################################################
############ Concatenate and prepare merge: ID_DATA and TEXT_DATA ######################
########################################################################################

############## Define a function to concatenate all csv files with file name that matches a certain pattern into one data frame
def concatenate (indir, file_name_match):
    os.chdir(indir)
    file_list = glob.glob(file_name_match)
    df_list = list()
    colnames = pd.read_csv(file_list[0], header = None).loc[0]
    
    for filename in file_list:
        # print(filename)
        df = pd.read_csv(filename, low_memory = False)
        df_list.append(df)

    df_concat = pd.concat(df_list, axis = 0)
    df_concat.columns = colnames
    return df_concat

############## Concatenate id_data and text_data files and create two data frames
text_data = concatenate('..\\filings', 'text_data_section_' + '*.csv')

############## Calculate tone : tone = (n_pos - n_negation - n_neg)/nw
text_data['tone_mda'] = (text_data['n_pos_mda'] - text_data['n_negation_mda'] - text_data['n_neg_mda'])/text_data['nw_mda']
text_data['tone_note'] = (text_data['n_pos_note'] - text_data['n_negation_note'] - text_data['n_neg_note'])/text_data['nw_note']

############## Save text_data dataframe into local file text_data_10-Q.csv
text_data.to_csv('..\\filings\\text_data_section.csv', index = 0)

print('Number of 10-Q sections parsed: ' + str(text_data.shape[0]))

Number of 10-Q sections parsed: 282391


In [3]:
########################################################################################
############### Merge SECTION_DATA with CRSP_COMPUSTAT_EDGAR_10-Q ######################
########################################################################################

crsp_comp_edgar = pd.read_csv(r'..\filings\crsp_comp_edgar_10-Q.csv')

############## prepare merge: ID_DATA
crsp_comp_edgar_section = pd.merge(crsp_comp_edgar, text_data, on = ['accnum'], how = 'inner', validate = '1:1')
print('number of observations after merging with section data: ' + str(crsp_comp_edgar_section.shape[0]))
crsp_comp_edgar_section

number of observations after merging with section data: 94211


,cusip,cik,rp,accnum,name,gvkey,SIC,fd,date_crsp,date_comp,cquarter,fyearq,fqtr,incorp,state,addzip,costat,age,actq,cheq,dpq,ibq,intanq,lctq,revtq,txditcq,xsgaq,atq,lag_atq,ceqq,lag_ceqq,cshoq,lag_cshoq,dlcq,lag_dlcq,dlttq,lag_dlttq,prccq,lag_prccq,ibq.1,lag_ibq,RET,STD_RET,EARN,LOSS,DEARN,STD_EARN,CFO,leap1_EARN,leap2_EARN,leap3_EARN,leap1_CFO,leap2_CFO,leap3_CFO,TACC,LAG_TA_REV,DSAR,PPE,SIC2,nw,nvocab,n_neg,n_pos,n_uctt,n_lit,n_cstr,n_modal_strong,n_modal_moderate,n_modal_weak,n_negation,tone,NW,TONE,TLAG,NEG,SIZE,MTB,LEV,AGE,nw_mda,n_neg_mda,n_pos_mda,n_negation_mda,nw_note,n_neg_note,n_pos_note,n_negation_note,tone_mda,tone_note
0,08373910,11454,1993-12-31,0000011454-94-000006,BERGEN BRUNSWIG CORP,2169.0,5122,1994-02-14,1993-12-31,1993-12-31,199312,1994.0,1.0,NJ,NJ,92868,I,10349,1474.257,22.616,7.381,10.331,NaN,1244.074,1834.936,2.278,81.343,2013.332,1772.337,417.292,417.800,35.577,36.431,2.026,2.129,337.515,285.873,17.875,17.50,10.331,-13.128,0.173416,0.082377,0.005829,0.0,0.013236,0.006858,-0.150724,0.007216,0.007869,0.008513,-0.047104,0.064367,0.002421,280.035,0.990116,0.005681,0.111794,51,6002,1234,104,30,37,184,24,11,1,15,0,-0.012329,8.700015,-12.329224,45,0,6.457621,1.525951,0.162498,9.244742,2294,13,17,0,1264,3,5,0,0.001744,0.001582
1,14428510,17843,1993-12-31,0000017843-94-000005,CARPENTER TECHNOLOGY CORP,2787.0,3312,1994-02-10,1993-12-31,1993-12-31,199312,1994.0,2.0,DE,DE,19103,A,20671,183.206,18.047,7.224,7.360,NaN,107.094,147.127,69.218,22.825,736.769,740.379,217.685,214.525,8.010,7.989,27.208,43.187,176.826,180.405,58.000,50.25,7.360,2.772,0.140644,0.044119,0.009941,0.0,0.006197,0.006358,0.052169,0.017407,0.021006,0.006757,0.022080,0.031095,-0.119186,-31.265,0.944874,0.027108,0.971353,33,31453,2980,385,98,197,1089,236,64,30,172,2,-0.009188,10.356281,-9.188313,41,0,5.995076,1.871331,0.301997,9.936535,402,3,2,0,2091,10,9,0,-0.002488,-0.000478
2,19247910,21510,1993-12-25,0000912057-94-000209,COHERENT INC,3157.0,3826,1994-02-07,1993-12-31,1993-12-31,199312,1994.0,1.0,DE,DE,95054,A,7701,141.164,36.470,NaN,1.429,NaN,50.855,47.026,1.643,20.341,189.397,193.796,119.437,117.023,10.101,9.913,7.847,9.498,12.015,14.122,12.750,14.25,1.429,2.250,-0.126372,0.029781,0.007374,0.0,-0.004236,0.003236,-0.105549,0.015607,0.014496,0.014704,0.023221,0.027511,0.003111,21.884,0.959793,-0.018138,0.387454,38,2310,685,27,16,15,13,7,3,5,1,3,-0.006061,7.745436,-6.060606,38,1,4.950604,1.207115,0.121881,8.949235,8,0,0,0,2037,27,20,3,0.000000,-0.004909
3,20172310,22444,1993-11-30,0000950134-94-000021,COMMERCIAL METALS CO,3246.0,3312,1994-01-14,1993-11-30,1993-11-30,199311,1994.0,1.0,DE,DE,75039,A,11477,376.632,33.843,7.186,5.723,NaN,197.216,381.760,14.773,25.409,526.641,541.961,240.087,235.421,11.082,11.061,64.375,45.827,74.565,76.737,35.000,37.75,5.723,7.509,-0.069489,0.030632,0.010560,0.0,-0.003295,0.003321,-0.135329,0.008112,0.012604,0.015347,-0.039636,0.030789,0.049424,79.066,0.967394,-0.047234,0.628757,33,1785,624,16,9,5,18,6,5,3,3,0,-0.003922,7.487734,-3.921569,45,1,6.034411,1.773643,0.226149,9.348187,6,0,0,0,1340,21,11,0,0.000000,-0.007463
4,24419910,315189,1994-01-31,0000315189-94-000009,DEERE & CO,3835.0,3523,1994-03-17,1994-01-31,1994-01-31,199401,1994.0,1.0,DE,DE,61265,A,22160,NaN,344.100,NaN,87.000,NaN,NaN,1726.800,8.700,258.000,11780.398,11351.898,2144.100,2085.399,85.801,85.502,2219.000,1603.400,2292.400,2548.500,80.500,77.25,87.000,109.500,0.023085,0.052329,0.007664,0.0,-0.001982,0.005185,-0.092883,0.016069,0.012624,0.013232,-0.004397,0.042722,0.035671,1141.400,1.055929,-0.057885,NaN,35,6219,1199,75,30,25,26,19,5,9,3,0,-0.007236,8.735525,-7.235890,45,0,8.795587,3.167274,0.365745,10.006089,3347,33,34,2,2190,27,7,0,-0.000299,-0.009132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [4]:
########################################################################################
############################### Variable Creation ######################################
########################################################################################

############################## Main Variables ##########################################
######## NW: natural log of 1 + total number of words in the document
crsp_comp_edgar_section['NW_MDA'] = np.log(1 + crsp_comp_edgar_section['nw_mda'])
crsp_comp_edgar_section['NW_NOTE'] = np.log(1 + crsp_comp_edgar_section['nw_note'])

######## TONE: number of net positive words (n_pos - n_neg - n_negations) per 1000 total words
crsp_comp_edgar_section['TONE_MDA'] = crsp_comp_edgar_section['tone_mda']*1000
crsp_comp_edgar_section['TONE_NOTE'] = crsp_comp_edgar_section['tone_note']*1000

In [5]:
########################################################################################
############################### Variable Screening #####################################
########################################################################################

## Change SIC to str
crsp_comp_edgar_section['SIC'] = crsp_comp_edgar_section['SIC'].astype(str)

########## Drop MDAs (firm-quarter) that contain number of words less than 1% threshold
# nwq01 = crsp_comp_edgar_section['nw_mda'].quantile(.01)
# print('number of words in MDA, 1% quantile: ' + str(nwq01))
del_word01 = crsp_comp_edgar_section.loc[crsp_comp_edgar_section['nw_mda'] < 100].shape[0]
print('number of MDAs that contain total words less than 1% threshold: ' + str(del_word01))
crsp_comp_edgar_section = crsp_comp_edgar_section.loc[crsp_comp_edgar_section['nw_mda'] >= 100]

########## Drop NOTES (firm-quarter) that contain number of words less than 1% threshold
# nwq01 = crsp_comp_edgar_section['nw_note'].quantile(.01)
# print('number of words in NOTES, 1% quantile: ' + str(nwq01))
del_word01 = crsp_comp_edgar_section.loc[crsp_comp_edgar_section['nw_note'] < 100].shape[0]
print('number of NOTES that contain total words less than 1% threshold: ' + str(del_word01))
crsp_comp_edgar_section = crsp_comp_edgar_section.loc[crsp_comp_edgar_section['nw_note'] >= 100]

############## Inspect sample size after variable screening
print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_section.shape[0]))

############## Save merged crsp_comp_edgar_section to csv file
crsp_comp_edgar_section.to_csv('..\\filings\\crsp_comp_edgar_section.csv', index = 0)

number of MDAs that contain total words less than 1% threshold: 12855
number of NOTES that contain total words less than 1% threshold: 1809
Number of firm-quarters after variable screening: 79547


In [6]:
########################################################################################
############### Table 1: Summary Statistics and Correlation Matrix #####################
########################################################################################

############# Table 1 Panel A: Summary statistics for selected variables
######### Variable groups:
# 1st line: textual variables, generally consistent with LM's summary statistics
# 2nd line: fundamental variables (main)
# 3rd line: abtone
selected_vars = crsp_comp_edgar_section[['NW','nw', 'NW_MDA','nw_mda', 'NW_NOTE','nw_note', 'TONE', 'TONE_MDA', 'TONE_NOTE', 'TLAG', \
                                   'RET', 'NEG', 'SIZE', 'MTB', 'LEV' \
                                 # 'AGE', 'age', 'EARN', 'STD_RET', 'STD_EARN', 'LOSS', 'DEARN' \
                                 ]]

T1PA = selected_vars.describe().transpose() 

############# Summary statistics for all raw and processed variables
full_summary = crsp_comp_edgar_section.describe().transpose()

T1PA

,count,mean,std,min,25%,50%,75%,max
NW,79547.0,9.229017,0.647696,7.049255,8.828201,9.261889,9.651366,13.490002
nw,79547.0,12501.477026,9423.660128,1151.000000,6823.000000,10528.000000,15542.000000,722159.000000
NW_MDA,79547.0,8.399241,0.641423,4.615121,8.059750,8.456594,8.814330,11.378822
nw_mda,79547.0,5299.066476,3161.529002,100.000000,3163.500000,4705.000000,6729.000000,87449.000000
NW_NOTE,79547.0,8.054006,0.888850,4.615121,7.546974,8.220672,8.681266,11.534403
nw_note,79547.0,4346.021660,3408.917529,100.000000,1894.000000,3716.000000,5890.500000,102170.000000
TONE,79547.0,-9.265521,6.776160,-60.727573,-13.182089,-8.067664,-4.416151,22.168480
TONE_MDA,79547.0,-5.294301,6.684179,-56.976744,-8.710871,-4.436557,-0.964436,26.845638
TONE_NOTE,79547.0,-9.778443,9.179078,-106.083086,-13.551780,-7.962434,-3.967208,41.450777
TLAG,79547.0,38.457918,6.156539,1.000000,35.000000,39.000000,43.000000,52.000000
